# CheckThat Task 2

The goal here is to find a method to extract a claim from a passage of text. For example:


- **Passage**: Hydrate YOURSELF W After Waking Up Water 30 min Before a Meal DRINK Before Taking a Shower →→ Before Going to Bed at the correct time T A YE Helps activate internal organs Helps digestion Helps lower blood pressure Helps to avoid heart attack Health+ by Punjab Kesari

- **Claim**: Drinking water at specific times can have different health benefits


To evaluate our method, we will use the **METEOR** metric on the **CLEF2025** dataset.




## Data Acquisition

The dataset will be a collection of text passage's and corresponding claims that have been extracted from the dataset. Let's go ahead and download the dataset

In [2]:
import os
from utils.data_utils import download

TRAIN_URL = "https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/raw/main/task2/data/train/train-eng.csv?inline=false"
TEST_URL = "https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/raw/main/task2/data/dev/dev-eng.csv?inline=false"

os.makedirs("data", exist_ok=True)
download(TRAIN_URL, "data")
download(TEST_URL, "data")

## Dataset

Now we make our dataset to hold the downloaded CSV data. This will allow us to iterate through our data easier. Each index of the dataset will return a text claim pair

In [4]:
from utils.dataset import ClaimVerificationDataset

train_dataset = ClaimVerificationDataset(f"data/train-eng.csv")
test_dataset = ClaimVerificationDataset(f"data/dev-eng.csv")

print(f"Train dataset length: {len(train_dataset)}")
print(f"Test dataset length: {len(test_dataset)}")

Train dataset length: 11374
Test dataset length: 1171


In [4]:
train_dataset.export_subset_to_csv("data/reduced-train-eng.csv")
test_dataset.export_subset_to_csv("data/reduced-test-eng.csv")

Subset successfully exported to data/reduced-train-eng.csv
Subset successfully exported to data/reduced-test-eng.csv


In [5]:
from utils.dataset import ClaimVerificationDataset

reduced_train_dataset = ClaimVerificationDataset(f"data/reduced-train-eng.csv")
reduced_test_dataset = ClaimVerificationDataset(f"data/reduced-test-eng.csv")

print(f"Reduced Train dataset length: {len(reduced_train_dataset)}")
print(f"Reduced Test dataset length: {len(reduced_test_dataset)}")

Reduced Train dataset length: 568
Reduced Test dataset length: 58


### Subset of Dataset for quick initial testing

## Method 1: Base Model Prompt Engineering

For this method, we will get a baseline score to see how well a model can do without finetuning

In [6]:
index = 80
print(train_dataset[index]["text"], "\n", train_dataset[index]["claim"])

Removing the death penalty for so-called "hate speech" is not enough. You must scrap the entire Bill, send it back to Singapore where it came from, get on your grubby and scabby knees and ask God and the Nigerian people for forgiveness for having the temerity to try to deprive them of their right to speak freely.

(Femi Fani-Kayode) Removing the death penalty for so-called "hate speech" is not enough. You must scrap the entire Bill, send it back to Singapore where it came from, get on your grubby and scabby knees and ask God and the Nigerian people for forgiveness for having the temerity to try to deprive them of their right to speak freely.

(Femi Fani-Kayode) Removing the death penalty for so-called "hate speech" is not enough. You must scrap the entire Bill, send it back to Singapore where it came from, get on your grubby and scabby knees and ask God and the Nigerian people for forgiveness for having the temerity to try to deprive them of their right to speak freely.

(Femi Fani-Kay

In [7]:
base_prompt = [
            {"role": "system", "content": "You are an AI assistant designed to extract claims from a given passage of text. Keep it short and return the claim in the text. Only return the big idea and exclude unneeded details."},
            {"role": "user", "content": train_dataset[0]["text"]},
            {"role": "assistant", "content": train_dataset[0]["claim"]},
            {"role": "user", "content": train_dataset[1]["text"]},
            {"role": "assistant", "content": train_dataset[1]["claim"]},
            {"role": "user", "content": train_dataset[15]["text"]},
            {"role": "assistant", "content": train_dataset[15]["claim"]},
            {"role": "user", "content": train_dataset[23]["text"]},
            {"role": "assistant", "content": train_dataset[23]["claim"]},
            {"role": "user", "content": train_dataset[29]["text"]},
            {"role": "assistant", "content": train_dataset[29]["claim"]},
        ]

In [12]:
print(base_prompt[1]['content'])
print(base_prompt[2]['content'])

print(base_prompt[3]['content'])
print(base_prompt[4]['content'])

print(base_prompt[5]['content'])
print(base_prompt[6]['content'])

print(base_prompt[7]['content'])
print(base_prompt[8]['content'])

print(base_prompt[9]['content'])
print(base_prompt[10]['content'])

Lieutenant Retired General Asif Mumtaz appointed as Chairman Pakistan Medical Commission PMC Lieutenant Retired General Asif Mumtaz appointed as Chairman Pakistan Medical Commission PMC Lieutenant Retired General Asif Mumtaz appointed as Chairman Pakistan Medical Commission PMC None
Pakistani government appoints former army general to head medical regulatory body
A priceless clip of 1970 of Bruce Lee playing Table Tennis with his Nan-chak !! His focus on speed A priceless clip of 1970 of Bruce Lee playing Table Tennis with his Nan-chak !! His focus on speed A priceless clip of 1970 of Bruce Lee playing Table Tennis with his Nan-chak !! His focus on speed None
Late actor and martial artist Bruce Lee playing table tennis with a set of nunchucks.
वीडियो को ज्यादा से ज्यादा शेयर करो 
जय श्री राम वीडियो को ज्यादा से ज्यादा शेयर करो 
जय श्री राम वीडियो को ज्यादा से ज्यादा शेयर करो 
जय श्री राम who feeds no friend
no man to love him
All guilt is he who
eats
alone
(so eat together
give and sha

In [16]:
test = (
        "Read the following social media posts. For each one, identify the main claim "
        "and rephrase it clearly and concisely.\n\n"
        "Example 1:\n"
        f"Post: {base_prompt[1]['content']}\n"
        f"Claim: {base_prompt[2]['content']}\n\n"
        "Example 2:\n"
        f"Post: {base_prompt[3]['content']}\n"
        f"Claim: {base_prompt[4]['content']}\n\n"
        "Example 3:\n"
        f"Post: {base_prompt[5]['content']}\n"
        f"Claim: {base_prompt[6]['content']}\n\n"
        "Example 4:\n"
        f"Post: {base_prompt[7]['content']}\n"
        f"Claim: {base_prompt[8]['content']}\n\n"
        "Example 5:\n"
        f"Post: {base_prompt[9]['content']}\n"
        f"Claim: {base_prompt[10]['content']}\n\n"
        f"Post: {"CLAIM HERE"}\n"
        "Claim:"
    )

In [17]:
print(test)

Read the following social media posts. For each one, identify the main claim and rephrase it clearly and concisely.

Example 1:
Post: Lieutenant Retired General Asif Mumtaz appointed as Chairman Pakistan Medical Commission PMC Lieutenant Retired General Asif Mumtaz appointed as Chairman Pakistan Medical Commission PMC Lieutenant Retired General Asif Mumtaz appointed as Chairman Pakistan Medical Commission PMC None
Claim: Pakistani government appoints former army general to head medical regulatory body

Example 2:
Post: A priceless clip of 1970 of Bruce Lee playing Table Tennis with his Nan-chak !! His focus on speed A priceless clip of 1970 of Bruce Lee playing Table Tennis with his Nan-chak !! His focus on speed A priceless clip of 1970 of Bruce Lee playing Table Tennis with his Nan-chak !! His focus on speed None
Claim: Late actor and martial artist Bruce Lee playing table tennis with a set of nunchucks.

Example 3:
Post: वीडियो को ज्यादा से ज्यादा शेयर करो 
जय श्री राम वीडियो को ज्य

In [ ]:
from agents.together_api_agent import TogetherAgent

together_agent = TogetherAgent()

In [7]:
from utils.metrics import evaluate_on_dataset

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jwilder\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
data, avg_score = evaluate_on_dataset(reduced_test_dataset, together_agent, base_prompt)

before added in 3 samples: 0.288298275862069

In [ ]:
print(avg_score)

# Method 2: LLM Finetuning

Please view testing.ipynb for the finetuning process

###########################################################################################################################################

In [1]:
from transformers import StoppingCriteria, StoppingCriteriaList
from unsloth import FastLanguageModel
import torch

class FinetunedAgent():
    def __init__(self):
        max_seq_length = 1024
        dtype = None
        load_in_4bit = True
        self.model, self.tokenizer = FastLanguageModel.from_pretrained(
            model_name="model-good/3B_finetuned_llama3.2",
            max_seq_length=max_seq_length,
            dtype=dtype,
            load_in_4bit=load_in_4bit
        )

        self.model = FastLanguageModel.for_inference(self.model)

    def ask(self, prompt: str):
        instruction = "You are an AI assistant designed to extract claims from a given passage of text. Keep it short and return the claim in the text. Only return the big idea and exclude unneeded details."
        
        class StopOnTokens(StoppingCriteria):
            def __init__(self, stop_ids):
                self.stop_ids = stop_ids
            
            def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
                for stop_id in self.stop_ids:
                    if input_ids[0][-1] == stop_id:
                        return True
                return False
        
        if not prompt or len(prompt.strip()) == 0:
            return "No input provided"
        
        inference_prompt = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

        ### Instruction:
        {instruction}

        ### Input:
        {prompt}

        ### Response:
        """
        
        try:
            inputs = self.tokenizer(
                [inference_prompt], 
                return_tensors="pt"
            ).to("cuda")
            
            
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=128,
                min_new_tokens=10,
                use_cache=True,
                temperature=0.7,
                do_sample=True,
                top_p=0.9,
                num_return_sequences=1,
                eos_token_id=self.tokenizer.eos_token_id,
                pad_token_id=self.tokenizer.pad_token_id or self.tokenizer.eos_token_id,
                stopping_criteria=[
                    StoppingCriteriaList([
                        StopOnTokens(
                            stop_ids=[
                                self.tokenizer.eos_token_id,
                                self.tokenizer.convert_tokens_to_ids("### Input:"),
                                self.tokenizer.convert_tokens_to_ids("### Instruction:")
                            ]
                        )
                    ])
                ]
            )
            
            full_response = self.tokenizer.batch_decode(outputs)[0]
            print("Full raw response:", full_response)
            
            response_parts = full_response.split("### Response:")
            
            if len(response_parts) > 1:
                answer = response_parts[1].split("###")[0].strip()
            else:
                answer = full_response.split("### Response:")[-1].strip()
            
            if not answer:
                answer = "No claim extracted"
            
            return answer
        
        except Exception as e:
            print(f"Error in claim extraction: {e}")
            return "Error during claim extraction"

C:\Users\jwilder\AppData\Local\Temp\ipykernel_96592\648366796.py:2: UserWarning: WARNING: Unsloth should be imported before transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.6.0+cu124 with CUDA 1204 (you have 2.6.0+cu118)
    Python  3.12.8 (you have 3.12.3)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
finetuned_agent = FinetunedAgent()

c:\Users\jwilder\home\projects\CheckThatLab\venv\Lib\site-packages\unsloth_zoo\gradient_checkpointing.py:330: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"cuda:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2025.3.18: Fast Llama patching. Transformers: 4.50.0.
   \\   /|    NVIDIA GeForce RTX 4060. Num GPUs = 1. Max memory: 7.996 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.6.0+cu118. CUDA: 8.9. CUDA Toolkit: 11.8. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.3.18 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [13]:
from tqdm import tqdm
from utils.metrics import evaluate_claim_extraction
from utils.dataset import ClaimVerificationDataset
import nltk
nltk.download('wordnet')
from nltk.translate import meteor
from nltk import word_tokenize

nltk.download('punkt_tab')

def evaluate_on_dataset_finetuned(test_dataset: ClaimVerificationDataset, agent: FinetunedAgent, limit = None):
    data = []
    scores = []

    counter = 0
    for entry in tqdm(test_dataset, desc="Evaluating LLM claim extraction"):
        user_prompt = entry["text"]
        output = agent.ask(user_prompt)
        meteor_score = evaluate_claim_extraction(entry["claim"], output)

        data.append({
            "ground_truth_claim": entry["claim"],
            "generated_claim": output,
            "meteor_score": meteor_score
        })

        scores.append(meteor_score)

        counter += 1
        if limit and counter >= limit:
            break

    avg_score = sum(scores) / len(scores) if scores else 0

    return data, avg_score

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jwilder\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\jwilder\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


In [14]:
data, avg_score = evaluate_on_dataset_finetuned(test_dataset, finetuned_agent, limit=5)

Evaluating LLM claim extraction:   0%|          | 0/1171 [00:00<?, ?it/s]

Full raw response: <|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

        ### Instruction:
        You are an AI assistant designed to extract claims from a given passage of text. Keep it short and return the claim in the text. Only return the big idea and exclude unneeded details.

        ### Input:
        The Karnofsky Jewish family, who immigrated to the United States from Lithuania, employed a 7-year-old boy and adopted (so to speak) him into their home.  He was originally given homework to get food because he was a starving kid.  He remained under the Jewish families employ, until he was 12  Karnofsky gave him money to buy his first instrument, which was a common instrument in Jewish families.  They really admired his musical talent.Later, when he became a professional … See More The Karnofsky Jewish family, who immigrated to the United States from Lit

Evaluating LLM claim extraction:   0%|          | 2/1171 [00:11<1:45:18,  5.41s/it]

Full raw response: <|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

        ### Instruction:
        You are an AI assistant designed to extract claims from a given passage of text. Keep it short and return the claim in the text. Only return the big idea and exclude unneeded details.

        ### Input:
        Trophy hunting is horrific. 😥😥😥 Trophy hunting is horrific. 😥😥😥 Trophy hunting is horrific. 😥😥😥 mill
Co
This little guy was rescued after his
mother was murdered by a hunter.
His eyes say it all, don't you think?
If you want to ban trophy hunting
forever, spread this everywhere.
Image Credit: FB. Changes: cropped, resized, text added. https://bit.ly/3b2vwas

        ### Response:
         Response: Response: Response: Response: Response: Response: Response: Response: Response: Response: Response: Response: Response: Response: Response: Response: Response

Evaluating LLM claim extraction:   0%|          | 3/1171 [00:15<1:37:12,  4.99s/it]

Full raw response: <|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

        ### Instruction:
        You are an AI assistant designed to extract claims from a given passage of text. Keep it short and return the claim in the text. Only return the big idea and exclude unneeded details.

        ### Input:
        Magarmacch // Heavy Rain //Hyderabad //Crocodile // Alert Magarmacch // Heavy Rain //Hyderabad //Crocodile // Alert Magarmacch // Heavy Rain //Hyderabad //Crocodile // Alert None

        ### Response:
         Response:
         Response:
         Response:
         Response:
         Response:
         Response:
         Response:
         Response:
         Response:
         Response:
         Response:
         Response:
         Response:
         Response:
         Response:
         Response:
         Response:
         Response:
         Respons

Evaluating LLM claim extraction:   0%|          | 4/1171 [00:20<1:33:52,  4.83s/it]

Full raw response: <|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

        ### Instruction:
        You are an AI assistant designed to extract claims from a given passage of text. Keep it short and return the claim in the text. Only return the big idea and exclude unneeded details.

        ### Input:
        The salary of a U.S. Senator is $174,000 per year. This is Joe Biden's house.... seems legit 🙄 The salary of a U.S. Senator is $174,000 per year. This is Joe Biden's house.... seems legit 🙄 The salary of a U.S. Senator is $174,000 per year. This is Joe Biden's house.... seems legit 🙄 None

        ### Response:
         None

        ### Response:
         None

        ### Response:
         None

        ### Response:
         None

        ### Response:
         None

        ### Response:
         None

        ### Response:
         None

        ##

Evaluating LLM claim extraction:   0%|          | 4/1171 [00:24<2:00:01,  6.17s/it]

Full raw response: <|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

        ### Instruction:
        You are an AI assistant designed to extract claims from a given passage of text. Keep it short and return the claim in the text. Only return the big idea and exclude unneeded details.

        ### Input:
        BREAKING NEWS | HUGE Explosion and chaos outside Kabul airport. Air strikes in Afghanistan are confirmed but whether that is US military or otherwise is not known. Update. 40 dead. 120 injured according to Italian military sources. Original images were NOT correct. Delete them. This Administration is now blaming the victims? Huh??? President Biden himself said he was surprised at the Taliban offensive. Hard to imagine the State Dept was as adamant as they say. BREAKING NEWS | HUGE Explosion and chaos outside Kabul airport. Air strikes in Afghanistan are 

In [15]:
print(avg_score)

0.01


The model seems to need more training. The responses seem to be struggling with the longer length text. With some shorter prompts I am able to get good results. Best results I have seen
with a fine tuned model so far is ~0.2

In [1]:
import torch
import os
import evaluate
import pandas as pd
import numpy as np



device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #make sure GPU is enabled.

import accelerate
import transformers
from datasets import load_dataset, DatasetDict, Dataset
from transformers import AutoTokenizer, AutoConfig, AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainingArguments
from torch.utils.data import DataLoader
from transformers import Seq2SeqTrainer

from nltk.tokenize import RegexpTokenizer
from torch.utils.data import DataLoader

from tqdm import tqdm

from utils.metrics import evaluate_claim_extraction




model_checkpoint = 'google/flan-t5-large'
model_code = model_checkpoint.split("/")[-1]

model = AutoModelForSeq2SeqLM.from_pretrained(f"./{model_code}/finetuned_{model_code}")
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jwilder\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
def testing():
    data = []
    scores = []

    counter = 0
    for entry in tqdm(test_dataset, desc="Evaluating LLM claim extraction"):
        #if counter == 10: break
        user_prompt = entry["text"]

        user_prompt = (
        "Please read the following social media post and extract the claim made within it. "
        "Normalize the claim by rephrasing it in a clear and concise manner.\n\n"
        f"Post: {user_prompt}\n\nExtracted Claim:"
        )

        #prompt.append({"role": "user", "content": user_prompt})
        #output = agent.ask(prompt)

        inputs = tokenizer(user_prompt, return_tensors="pt", padding=True, truncation=True, max_length=128)
        model.eval()
        with torch.no_grad():  # Disable gradient calculation
            generated_ids = model.generate(inputs["input_ids"], max_length=128, num_beams=5, early_stopping=True)

        output = tokenizer.decode(generated_ids[0], skip_special_tokens=True)


        meteor_score = evaluate_claim_extraction(entry["claim"], output)

        data.append({
            "ground_truth_claim": entry["claim"],
            "generated_claim": output,
            "meteor_score": meteor_score
        })

        scores.append(meteor_score)
        counter += 1


    avg_score = sum(scores) / len(scores) if scores else 0

    return data, avg_score

In [5]:
data, avg_score = testing()

Evaluating LLM claim extraction: 100%|██████████| 1171/1171 [52:22<00:00,  2.68s/it]


In [6]:
print(avg_score)

0.5568712211784799


In [7]:
for entry in data:
    print(f"ground truth: {entry['ground_truth_claim']}")
    print(f"generated: {entry['generated_claim']}")
    print(f"meteor: {entry['meteor_score']}\n")

ground truth: Photo shows Louis Armstrong as a child
generated: A Holocaust story told in the New York Times
meteor: 0.0694

ground truth: This leopard cub's mother was killed by a trophy hunter
generated: This leopard cub's mother was killed by a trophy hunter
meteor: 0.9996

ground truth: Videos show current situation of Hyderabad amid heavy rain
generated: Video shows a crocodile spotted in a residential area in Hyderabad during the ongoing heavy rains.
meteor: 0.4551

ground truth: Joe Biden lives in a large estate bought on a senator's salary
generated: Joe Biden owns the house in this photo.
meteor: 0.2028

ground truth: Photo shows August 26, 2021 explosion near Kabul airport
generated: A photo of an explosion at an airport in Afghanistan is authentic
meteor: 0.1471

ground truth: White people own only 22 percent of South Africa's land
generated: White people own only 22 percent of South Africa’s land
meteor: 0.8973

ground truth: This video shows Amitabh Bachchan thanking healt

In [8]:
print(avg_score)

0.5568712211784799


In [9]:
import csv
def dictionary_to_csv(data: list, filename: str):
    if not data:
        raise ValueError("Data list is empty")
    
    fieldnames = ['ground_truth_claim', 'generated_claim', 'meteor_score']
    
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)

dictionary_to_csv(data, "finetunedflant5-output.csv")